In [1]:
# 1. import the libaries for the project
import pandas as pd
import sqlite3

In [2]:
# create a variable to the path of the sqlite db
# to make it easier keep file in sale folder as the notebook
DB_PATH = "thrive_test_db.db"

In [3]:
# connect to the database and add a timeout of 10seconds
conn = sqlite3.connect(DB_PATH, timeout=10)
cursor = conn.cursor()

In [4]:
# create a sql variable to help view the data in the table users
query = "SELECT * FROM users"

In [5]:
# use pandas to read the data
df = pd.read_sql_query(query,conn)

In [6]:
# you can choose to export the data to cvs file to further view
# not the best if data is huge
df.to_csv('users.csv')

In [7]:
df

,name,email,id,is_customer
0,Randy Young,randy@example.com,0,1
1,Michael Cox,michael@example.com,1,1
2,Raymond Cox,raymond@example.com,2,1
3,Mary Phillips,mary@example.com,3,1
4,Kelly Garza,kelly@example.com,4,1
5,David Norton,david@example.com,5,1
6,John Taylor,john@example.com,6,0
7,Anita Wang,anita@example.com,7,1
8,Deanna Munoz,deanna@example.com,8,0
9,Kelly Mcmillan,kelly@example.com,9,1


In [8]:
query = "SELECT * FROM conversation_start"

In [9]:
df = pd.read_sql_query(query,conn)

In [10]:
df.to_csv('conversation_start.csv')

In [11]:
df

,created_at,source_type,id,priority,conv_dataset_email,message
0,1679498647,conversation,10065,not_priority,kelly@example.com,Show dog box. Catch act over medical ahead exe...
1,1679513798,conversation,10106,not_priority,anita@example.com,Everyone never government forget wall too. Mor...
2,1679602052,conversation,10222,not_priority,david@example.com,Audience present end field public. Resource ca...
3,1679688990,conversation,10308,not_priority,kelly@example.com,Operation hot condition however particularly p...
4,1679865951,conversation,10342,not_priority,karen@example.com,Suffer quickly officer trade service. Mission ...
5,1725885557,conversation,100120,not_priority,michael@example.com,Yard give enter black. Issue use group. Gun ra...
6,1726167821,conversation,102013,not_priority,mary@example.com,Election run myself pretty fill onto. Break em...
7,1726447265,conversation,102837,not_priority,nicholas@example.com,Act information performance truth strong desig...
8,1679875562,conversation,10350,not_priority,raymond@example.com,Buy gun smile arm late. Although shoulder resu...
9,1679941011,conversation,10404,not_priority,deanna@example.com,Organization society sometimes perhaps door. M...


In [12]:
query = "SELECT * FROM conversation_parts"

In [13]:
df = pd.read_sql_query(query,conn)

In [14]:
df

,created_at,conversation_id,part_type,id,conv_dataset_email,message
0,1679498651,10065,comment,20482598151,john@example.com,Show dog box. Catch act over medical ahead exe...
1,1679498716,10065,assignment,20482622915,deanna@example.com,Everyone never government forget wall too. Mor...
2,1679498908,10065,comment,20482707054,kelly@example.com,Audience present end field public. Resource ca...
3,1679499097,10065,comment,20482788524,deanna@example.com,Operation hot condition however particularly p...
4,1679499376,10065,comment,20482911573,kelly@example.com,Suffer quickly officer trade service. Mission ...
...,...,...,...,...,...,...
80,1726494982,102837,open,21904986991,nicholas@example.com,Together physical discussion design. Happy rec...
81,1726495021,102837,comment,21904986992,deanna@example.com,Do community side hour edge father cell. City ...
82,1726495022,102837,close,21904986994,deanna@example.com,Range day all especially be environmental. Kin...
83,1726495252,102837,open,21904986998,nicholas@example.com,Consumer health present beat simply visit. Awa...


In [15]:
df.to_csv('conversation_parts.csv')

In [16]:
# 2. Create Schemas
# 2a. This create User Dimension table named dim_user
cursor.execute("""
CREATE TABLE IF NOT EXISTS dim_user (
    user_id      INTEGER PRIMARY KEY,
    email        VARCHAR,
    name         TEXT,
    is_customer  BOOLEAN
)
""")
conn.commit()

In [17]:
# 2b. This create Conversation Parts Dimension table named dim_conversation_part
cursor.execute("""
CREATE TABLE IF NOT EXISTS dim_conversation_part (
    part_id            INTEGER PRIMARY KEY,
    conversation_id    INTEGER,
    conv_dataset_email VARCHAR,
    part_type          TEXT,
    message            TEXT,
    created_at         DATETIME
)
""")
conn.commit()

In [18]:
# 2c. Create the Fact Table named Consolidated_messages table named consolidated_messages
cursor.execute("""
CREATE TABLE IF NOT EXISTS consolidated_messages (
    id              INTEGER,
    user_id         INTEGER,
    email           VARCHAR,
    conversation_id INTEGER,
    message         TEXT,
    message_type    TEXT,
    created_at      DATETIME,
    FOREIGN KEY(user_id) REFERENCES dim_user(user_id)
)
""")
conn.commit()

In [19]:
# 3. Populate Dimensions with separate commits
# 3a. dim_user
cursor.execute("DELETE FROM dim_user;")
conn.commit()

In [20]:
cursor.execute(
    "INSERT INTO dim_user (user_id, email, name, is_customer) "
    "SELECT id, email, name, is_customer FROM users;"
)
conn.commit()

In [21]:
query = "SELECT * FROM dim_user"

In [22]:
df = pd.read_sql_query(query,conn)

In [23]:
df

,user_id,email,name,is_customer
0,0,randy@example.com,Randy Young,1
1,1,michael@example.com,Michael Cox,1
2,2,raymond@example.com,Raymond Cox,1
3,3,mary@example.com,Mary Phillips,1
4,4,kelly@example.com,Kelly Garza,1
5,5,david@example.com,David Norton,1
6,6,john@example.com,John Taylor,0
7,7,anita@example.com,Anita Wang,1
8,8,deanna@example.com,Deanna Munoz,0
9,9,kelly@example.com,Kelly Mcmillan,1


In [24]:
cursor.execute("DELETE FROM dim_conversation_part;")
conn.commit()

In [25]:
cursor.execute(
    "INSERT INTO dim_conversation_part (part_id, conversation_id, conv_dataset_email, part_type, message, created_at) "
    "SELECT id, conversation_id, conv_dataset_email, part_type, message, created_at FROM conversation_parts;"
)
conn.commit()

In [26]:
cursor.execute("DELETE FROM consolidated_messages;")
conn.commit()

In [27]:
# 4b. Insert consolidated rows
cursor.execute("""
INSERT INTO consolidated_messages (id, user_id, email, conversation_id, message, message_type, created_at)
WITH
  start_msgs AS (
    SELECT
      id           AS msg_id,
      conv_dataset_email AS email,
      id           AS conversation_id,
      message,
      'open'       AS message_type,
      created_at
    FROM conversation_start
  ),
  part_msgs AS (
    SELECT
      part_id           AS msg_id,
      conv_dataset_email AS email,
      conversation_id,
      message,
      part_type    AS message_type,
      created_at
    FROM dim_conversation_part
  ),
  all_msgs AS (
    SELECT * FROM start_msgs
    UNION ALL
    SELECT * FROM part_msgs
  ),
  conv_customer AS (
    SELECT
      am.conversation_id,
      u.user_id AS user_id
    FROM all_msgs am
    JOIN dim_user u
      ON am.email = u.email
    WHERE u.is_customer = 1
    GROUP BY am.conversation_id, u.user_id
  )
SELECT
  am.msg_id      AS id,
  cc.user_id     AS user_id,
  am.email       AS email,
  am.conversation_id,
  am.message,
  am.message_type,
  am.created_at
FROM all_msgs am
JOIN conv_customer cc
  ON am.conversation_id = cc.conversation_id
ORDER BY
  am.conversation_id,
  am.created_at;
""")
conn.commit()

In [28]:
query = "SELECT * FROM consolidated_messages"

In [29]:
df = pd.read_sql_query(query,conn)

In [30]:
df

,id,user_id,email,conversation_id,message,message_type,created_at
0,10065,4,kelly@example.com,10065,Show dog box. Catch act over medical ahead exe...,open,1679498647
1,10065,9,kelly@example.com,10065,Show dog box. Catch act over medical ahead exe...,open,1679498647
2,20482598151,4,john@example.com,10065,Show dog box. Catch act over medical ahead exe...,comment,1679498651
3,20482598151,9,john@example.com,10065,Show dog box. Catch act over medical ahead exe...,comment,1679498651
4,20482622915,4,deanna@example.com,10065,Everyone never government forget wall too. Mor...,assignment,1679498716
...,...,...,...,...,...,...,...
107,21904986991,10,nicholas@example.com,102837,Together physical discussion design. Happy rec...,open,1726494982
108,21904986992,10,deanna@example.com,102837,Do community side hour edge father cell. City ...,comment,1726495021
109,21904986994,10,deanna@example.com,102837,Range day all especially be environmental. Kin...,close,1726495022
110,21904986998,10,nicholas@example.com,102837,Consumer health present beat simply visit. Awa...,open,1726495252


In [31]:
df.to_csv('consolidated_messages.csv')

In [32]:
conn.close()